<a href="https://colab.research.google.com/github/gi3265/-Kaggle-Disaster_Tweets/blob/hyebin/%EC%BA%90%EA%B8%80_%ED%8A%B8%EC%9C%84%ED%84%B0%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import pandas as pd
import numpy as np

In [7]:
os.chdir('/content/drive/MyDrive/캐글트위터')
os.getcwd()

'/content/drive/MyDrive/캐글트위터'

In [8]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
for i in range(train.shape[0]):
  print(i, ': ' , train.loc[i, 'text'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3235 :  Our doctors and nurses in the new Pediatric Emergency Department are all specialized in child services! http://t.co/k1TMLWvjmJ
3236 :  #MissionHills CA #Nursing : Registered Nurse - Emergency Department ( Full Time... at Providence Health &amp; Services http://t.co/Z5grLREy6V
3237 :  Just saw a car on the I-77 Fully engulfed in flames hahah
3238 :  Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/P6cyLz5lpt http://t.co/Jpu9gIps9f
3239 :  Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/CYzlshlQhG http://t.co/nDiS8f1vzt
3240 :  He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam
3241 :  #TRAFFICALERT  Eastbound 210 Freeway at Citrus Ave in Azusa. Two motorcycles involved in accident with one fully engulfed in flames in lanes
3242 :  Men escape car engulfed in flames in Parley's Canyon c

# 1. 전처리

In [11]:
!pip install pyspellchecker

     |████████████████████████████████| 2.7MB 18.0MB/s 


In [12]:
import re

# url 제거
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


# html 제거
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


# 이모티콘 제거
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# 영어 외 제거
def only_eng(text):
  only = re.sub('[^a-zA-Z\']', ' ', text)
  return only.lower()



# 오타 고쳐줌 -> 성능 좋은지 잘 모르겠음
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [19]:
train['text2'] = train['text'].apply(remove_URL)
train['text2'] = train['text2'].apply(remove_html)
train['text2'] = train['text2'].apply(remove_emoji)
train['text2'] = train['text2'].apply(only_eng)

In [ ]:
from tqdm.notebook import tqdm

for i in tqdm(range(train.shape[0])):
  train.loc[i, 'text2'] = correct_spellings(train.loc[i, 'text2'])

In [21]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
train['text2'] = train['text2'].apply(lambda x: x.split())

In [15]:
# 불용어 제거
stops = set(stopwords.words('english'))

def remove_stop(text):
  return [word for word in text if not word in stops]


# 어간 추출 -> 성능 좋은지 잘 모르겠음
stemmer = nltk.stem.SnowballStemmer('english')
def interval(text):
  return [stemmer.stem(word) for word in text]

In [16]:
train['text2'] = train['text2'].apply(remove_stop)
# train['text2'] = train['text2'].apply(interval)

In [22]:
for i in range(train.shape[0]):
  print(i, ': ' , train.loc[i, 'text2'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2613 :  how can we help save a beautiful town in ontario from destruction by a power plant developer  
2614 :  crackdown   destruction restricted to multiplayer  crackdown   impressed earlier this week with a demonstratio    
2615 :   danhrothschild greed is the fuel of self destruction   takecare
2616 :   cinla      windowgatribble the contrasts of foreboding destruction enhanced by expansive divisions of color saturation contrast and hue 
2617 :  russian authorities to take account of petition against destruction of sanctioned food  vladimir putin's press    
2618 :  fall back this first break homebuyer miscalculation that could destruction thousands  mwjcdk
2619 :  there's a  fly loose in my workspace with two  bored  cats  i forsee terrible things    destruction  badkitty  thisiswhywecanthavenicethings
2620 :   lt dave so many specs so much fan service so much lore destruction
2621 :  truck driver salvages banned tomatoes from destruction on  rus

# 2. Fasttext 사용

In [30]:
for i in range(train.shape[0]):
    train.loc[i, 'text3'] = '__label__' + str(train.loc[i, 'target']) + '   ' + train.loc[i, 'text2']

train.head()

,id,keyword,location,text,target,text2,text3
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,__label__1 our deeds are the reason of this ...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la range ask canada,__label__1 forest fire near la range ask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,__label__1 all residents asked to shelter in...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,__label__1 people receive wildfires evacuati...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,__label__1 just got sent this photo from rub...


In [ ]:
train.text3.to_csv('./fasttext_data.txt', index = False, header = False)

In [41]:
train.text2.to_csv('./fasttext_data2.txt', index = False, header = False)

In [3]:
pip install fasttext

     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093204 sha256=3c083830effc1770bb5c2a153bfad905386cbb619340a87987be656425a610e7
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


## 2-1. unsupervised

In [42]:
import fasttext

model = fasttext.train_unsupervised('./fasttext_data2.txt',model='skipgram', epoch=5, lr = 0.1)
print(model.words)   

['</s>', 'the', 'a', 'in', 'to', 'of', 'i', 'and', 'is', 'for', 'on', 'you', 'my', 'it', 'with', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'amp', 'as', 'up', 'me', 'just', 'but', 'so', 'not', 'your', 'out', 'no', 'after', 'all', 'will', 'when', 'an', 'has', 'fire', "i'm", 'we', 'if', 'get', 'new', 'now', 's', 'via', 'more', "'", 'about', 'news', 'or', 'one', 'he', 'people', 'they', 'what', 'been', 'how', 'over', 'who', "it's", 'into', "don't", 'do', 'video', 'can', 'emergency', 'disaster', 'there', 'police', 'than', 'u', 'her', 'time', 'would', 'his', 'still', 'some', 'us', 'body', 'were', 'burning', 'back', 'day', 'storm', 'crash', 'off', 'got', 'california', 'them', 'why', 'know', 'man', 'suicide', 'had', 'buildings', 'rt', 'first', 'see', 'bomb', 'going', 'nuclear', 'pm', 'world', 'two', 'love', 'fires', 'our', 'killed', 'year', 'youtube', 'w', 'go', 'attack', 'dead', 'full', 'their', 'gt', 'train', 'life', 'hiroshima', 'car', 'war', 'here', 'old', 'tod

In [43]:
print(model['people'])

[ 0.04454753  0.40927115 -0.01809011 -0.10495487  0.04713496  0.30494094
  0.09858505 -0.43187252  0.04795912  0.11205859  0.1508652   0.39443454
  0.25106162  0.18047886  0.16689132 -0.287551   -0.16485609  0.23299517
  0.04531436 -0.09326167 -0.08652434  0.103658    0.27297384 -0.06209563
 -0.19152679 -0.054885    0.17931949 -0.03817885 -0.18117736  0.07516804
  0.12783694 -0.07363957 -0.08090326 -0.0312335  -0.2358464  -0.3197194
 -0.04903391  0.02709189 -0.19102015  0.30243295  0.01988512  0.08181808
  0.21335869  0.17609514 -0.03462186  0.00114478  0.15455814 -0.17213818
  0.01447495 -0.14600453 -0.03505776 -0.12939832 -0.20751534  0.07546713
  0.0679481  -0.09470098 -0.17178023  0.18365957  0.30845547  0.0791052
  0.02226769  0.05561144  0.05863367 -0.0297962   0.21875946  0.01329163
 -0.15863958  0.04793463  0.3076604   0.39609644 -0.23508765  0.199297
  0.1266567   0.01237786  0.00045585 -0.09424613  0.19221094  0.01174541
  0.14361973 -0.11613762 -0.05603511  0.0520842   0.054

In [45]:
for i in range(len(model.words)):
    if 'lover' in model.words:
        print(i)

In [46]:
model.get_word_vector('lover') # 포함되어 있지 않은 단어(oov)에서도 적용 가능

array([-0.06583993,  0.40717664,  0.19064897,  0.05389549, -0.04971483,
        0.13095583, -0.00833362, -0.31479734,  0.29145658,  0.12095639,
       -0.01407902, -0.06510019,  0.29375732,  0.011755  , -0.03364199,
       -0.04362453, -0.18552776,  0.11398912,  0.05689664,  0.11429385,
        0.24026848,  0.3579223 ,  0.40320987, -0.13155052,  0.06124631,
       -0.11836613,  0.06943502,  0.17650011, -0.08823319,  0.19600017,
       -0.0615651 , -0.08350693, -0.05581418, -0.00589611, -0.34530574,
       -0.37554947, -0.05468163,  0.26295912, -0.46165192,  0.29667106,
        0.04308964, -0.09302894,  0.1872167 ,  0.13052535,  0.13610104,
       -0.00929963,  0.16247015, -0.07213228, -0.04172582, -0.15975833,
        0.02466149, -0.12067932, -0.11519555,  0.48999935,  0.24844588,
       -0.00073138, -0.13061637,  0.24496861,  0.03075686,  0.30735454,
       -0.10338689,  0.09497481,  0.04650926, -0.1598237 ,  0.00164838,
        0.03689711,  0.04129257, -0.0185586 ,  0.21239617,  0.03

## 2-2. supervised

In [51]:
classifier = fasttext.train_supervised('fasttext_data.txt', label_prefix='__label__')

In [52]:
texts = ['example very long text 1', 'example very long text 2']
labels = classifier.predict(texts)
print (labels)

([['__label__0'], ['__label__0']], [array([0.991352], dtype=float32), array([0.991352], dtype=float32)])


In [53]:
texts = ['An earthquake occurred', 'run away']
labels = classifier.predict(texts)
print (labels) ## 스스로 라벨도 파악함

([['__label__1'], ['__label__0']], [array([0.986103], dtype=float32), array([0.8838451], dtype=float32)])
